In [1]:
import os
import time
import json
import tqdm
import logging
import geocoder
import pandas as pd
from pprint import pprint

In [2]:
BAIDU_AK = '62c28637e66010289bdef04310267902'
COUNTRY_JSON_FILENAME = 'country.json'
PROVINCES_JSON_FILENAME = 'provinces.json'

LOG_FILENAME = "lastlog.log"
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s\n"
with open(LOG_FILENAME, 'w') as f:
    f.write(LOG_FORMAT)
logging.basicConfig(filename=LOG_FILENAME, level=logging.DEBUG, format=LOG_FORMAT)

In [3]:
last_ncov_json = sorted([e for e in os.listdir('.') if e.startswith('ncov_')], reverse=True)[0]
print(last_ncov_json)
country_df = pd.read_json(last_ncov_json)
country_df

ncov_20200207-092154.json


,provinceName,provinceShortName,confirmedCount,suspectedCount,curedCount,deadCount,comment,locationId,cities
0,湖北省,湖北,22112,0,817,618,,420000,"[{'cityName': '武汉', 'confirmedCount': 11618, '..."
1,浙江省,浙江,1006,0,98,0,待明确地区：治愈 4,330000,"[{'cityName': '温州', 'confirmedCount': 421, 'su..."
2,广东省,广东,970,0,69,0,,440000,"[{'cityName': '深圳', 'confirmedCount': 316, 'su..."
3,河南省,河南,914,0,68,3,参考卫健委统计口径，部分县区与地级市合并,410000,"[{'cityName': '信阳', 'confirmedCount': 176, 'su..."
4,湖南省,湖南,711,0,81,0,,430000,"[{'cityName': '长沙', 'confirmedCount': 179, 'su..."
5,安徽省,安徽,665,0,34,0,,340000,"[{'cityName': '合肥', 'confirmedCount': 115, 'su..."
6,江西省,江西,661,0,45,0,,360000,"[{'cityName': '南昌', 'confirmedCount': 162, 'su..."
7,重庆市,重庆,411,0,24,2,,500000,"[{'cityName': '万州区', 'confirmedCount': 73, 'su..."
8,江苏省,江苏,408,0,38,0,,320000,"[{'cityName': '苏州', 'confirmedCount': 69, 'sus..."
9,山东省,山东,379,0,31,0,,370000,"[{'cityName': '青岛', 'confirmedCount': 43, 'sus..."


In [4]:
provinces_df = pd.DataFrame()
for idx in country_df.index:
    se = country_df.iloc[idx]
    if se['cities']:
        provinceName = se['provinceName']
        cities_df = pd.read_json(json.dumps(se['cities']))
        with tqdm.tqdm(cities_df.index) as pbar:
            for idx in pbar:
                se = cities_df.iloc[idx].copy()
                try:
                    pbar.set_description('{} {}'.format(provinceName, se['cityName']))
                    g = geocoder.baidu(u"{} {}".format(provinceName, se['cityName']), key=BAIDU_AK)
                    se['provinceName'] = provinceName
                    se['lat'] = g.latlng[0]
                    se['lng'] = g.latlng[1]
                    provinces_df = provinces_df.append(se, ignore_index=True)
                except KeyError as e:
                    logging.error('{}-{}-{}'.format(repr(e), provinceName, se['cityName']))
                    print(repr(e), provinceName, se['cityName'])

海南省 陵水:  50%|███████████████               | 8/16 [00:01<00:00,  8.51it/s]     

KeyError('result') 海南省 未知地区


西藏自治区 拉萨: 100%|███████████████████████████| 1/1 [00:00<00:00, 11.34it/s]       


In [5]:
provinces_df

,cityName,confirmedCount,curedCount,deadCount,lat,lng,locationId,provinceName,suspectedCount
0,武汉,11618.0,496.0,478.0,30.598467,114.311582,420100.0,湖北省,0.0
1,孝感,2141.0,9.0,25.0,30.930689,113.922510,420900.0,湖北省,0.0
2,黄冈,1897.0,65.0,32.0,30.459359,114.878490,421100.0,湖北省,0.0
3,随州,915.0,9.0,9.0,31.696517,113.389450,421300.0,湖北省,0.0
4,荆州,885.0,18.0,10.0,30.340842,112.245523,421000.0,湖北省,0.0
...,...,...,...,...,...,...,...,...,...
408,五家渠,1.0,0.0,0.0,44.172445,87.549937,659004.0,新疆维吾尔自治区,0.0
409,巴州,1.0,0.0,0.0,31.857686,106.775513,511902.0,新疆维吾尔自治区,0.0
410,西宁,15.0,3.0,0.0,36.623385,101.784450,630100.0,青海省,0.0
411,海北州,3.0,0.0,0.0,36.960663,100.907434,632200.0,青海省,0.0


In [6]:
country_df.to_json(open(COUNTRY_JSON_FILENAME, 'w'))
provinces_df.to_json(open(PROVINCES_JSON_FILENAME, 'w'))